### Adjust Word Group Repeat With Youtube Link

In [34]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [35]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [58]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 5000  # 28 native word end index

# file extention
file_ext = "Fivegram"

In [59]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [60]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 3/Result/3-Adjust Word Group Repeat With Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

##### Link Count

In [61]:
path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 3/Result/2-Find Word Group Youtube Link"
file = f"{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Selected_Manuel.xlsx"
sheet = "Sheet1"  # Sheet1

In [64]:
df_link_selected_manuel = pd.read_excel(f"{path_folder}/{file}", sheet_name=f"{sheet}")
df_link_selected_manuel

,search_string,start_time,end_time,video_id
0,başka bir şey var,816,818,j1jgprjyYxw
1,başka bir şey yok,775,777,g81mNCrBnO4
2,başka bir şey yok,145,147,eGGS2bYtnLE
3,başka bir şey yok,1706,1708,UJBXM9wxjlI
4,başka bir şey yok,2650,2652,8_VkxmHeMAk
5,başka bir şey yok,1009,1011,6XgenY_eEro
6,başka bir şey yok,5576,5578,oE9qrxR7DpI
7,başka bir şey yok,2346,2349,UtcBPDTBiko
8,ben de öyle düşünmüştüm,6569,6572,B2BcTrXcsVE
9,ben de öyle düşünmüştüm,2215,2217,yP0QLRg5DWc


In [65]:
df_link_selected_manuel["video_url"] = "https://www.youtube.com/watch?v="+df_link_selected_manuel['video_id'].map(str)+"&t="+df_link_selected_manuel['start_time'].map(str)+"s"
df_link_selected_manuel

,search_string,start_time,end_time,video_id,video_url
0,başka bir şey var,816,818,j1jgprjyYxw,https://www.youtube.com/watch?v=j1jgprjyYxw&t=...
1,başka bir şey yok,775,777,g81mNCrBnO4,https://www.youtube.com/watch?v=g81mNCrBnO4&t=...
2,başka bir şey yok,145,147,eGGS2bYtnLE,https://www.youtube.com/watch?v=eGGS2bYtnLE&t=...
3,başka bir şey yok,1706,1708,UJBXM9wxjlI,https://www.youtube.com/watch?v=UJBXM9wxjlI&t=...
4,başka bir şey yok,2650,2652,8_VkxmHeMAk,https://www.youtube.com/watch?v=8_VkxmHeMAk&t=...
5,başka bir şey yok,1009,1011,6XgenY_eEro,https://www.youtube.com/watch?v=6XgenY_eEro&t=...
6,başka bir şey yok,5576,5578,oE9qrxR7DpI,https://www.youtube.com/watch?v=oE9qrxR7DpI&t=...
7,başka bir şey yok,2346,2349,UtcBPDTBiko,https://www.youtube.com/watch?v=UtcBPDTBiko&t=...
8,ben de öyle düşünmüştüm,6569,6572,B2BcTrXcsVE,https://www.youtube.com/watch?v=B2BcTrXcsVE&t=...
9,ben de öyle düşünmüştüm,2215,2217,yP0QLRg5DWc,https://www.youtube.com/watch?v=yP0QLRg5DWc&t=...


In [66]:
df_count = df_link_selected_manuel.groupby(["search_string"])[["search_string"]].count()
df_count.rename(columns={"search_string":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

,search_string,count
0,başka bir şey yok,7
1,bir fikrin var mı,7
2,ne var ne yok,7
3,ben de seni seviyorum,5
4,bir şey sorabilir miyim,5
5,bir şey mi var,4
6,burada ne işin var,4
7,ne olduğunu biliyor musun,4
8,önemli bir şey değil,4
9,ben de öyle düşünmüştüm,3


In [67]:
df_count = df_count[df_count["count"] > 2]

In [68]:
df_count_merge = pd.merge(df_link_selected_manuel, df_count, how="inner",on= "search_string")
df_count_merge.sort_values(by=["count","search_string"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

,search_string,start_time,end_time,video_id,video_url,count
0,ne var ne yok,208,210,KjOlhIa9X2Y,https://www.youtube.com/watch?v=KjOlhIa9X2Y&t=...,7
1,ne var ne yok,2182,2184,XWd602XfKh0,https://www.youtube.com/watch?v=XWd602XfKh0&t=...,7
2,ne var ne yok,3000,3002,OckqrD0x_r8,https://www.youtube.com/watch?v=OckqrD0x_r8&t=...,7
3,ne var ne yok,2763,2765,MuqJeFdzXgE,https://www.youtube.com/watch?v=MuqJeFdzXgE&t=...,7
4,ne var ne yok,6826,6828,94KUKlC1rlc,https://www.youtube.com/watch?v=94KUKlC1rlc&t=...,7
5,ne var ne yok,97,99,S6wCHPZLjsw,https://www.youtube.com/watch?v=S6wCHPZLjsw&t=97s,7
6,ne var ne yok,5868,5870,cslXqrcH5qg,https://www.youtube.com/watch?v=cslXqrcH5qg&t=...,7
7,bir fikrin var mı,2480,2482,lNwXQrpMfhM,https://www.youtube.com/watch?v=lNwXQrpMfhM&t=...,7
8,bir fikrin var mı,5895,5897,sMEBFgBFmy8,https://www.youtube.com/watch?v=sMEBFgBFmy8&t=...,7
9,bir fikrin var mı,1318,1320,cXAaYu0JM_s,https://www.youtube.com/watch?v=cXAaYu0JM_s&t=...,7


In [69]:
df_count.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 3/Data/Deployment/Talk Time 3 {file_ext} Video List.xlsx", index=False)

In [70]:
df_count_merge.to_excel(f"{path_folder}/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Selected_Count_Manuel.xlsx", index=False)

##### Link Repeat

In [71]:
path_folder_file2 = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 3/Result/2-Find Word Group Youtube Link/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Selected_Count_Manuel.xlsx"
sheet2 = "Sheet1"  # Sheet1

In [72]:
df_word_group_time_loc = pd.read_excel(f"{path_folder_file2}", sheet_name=f"{sheet2}")
#df_word_group_time_loc = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Manuel.xlsx")   
df_word_group_time_loc

,search_string,start_time,end_time,video_id,video_url,count
0,ne var ne yok,208,210,KjOlhIa9X2Y,https://www.youtube.com/watch?v=KjOlhIa9X2Y&t=...,7
1,ne var ne yok,2182,2184,XWd602XfKh0,https://www.youtube.com/watch?v=XWd602XfKh0&t=...,7
2,ne var ne yok,3000,3002,OckqrD0x_r8,https://www.youtube.com/watch?v=OckqrD0x_r8&t=...,7
3,ne var ne yok,2763,2765,MuqJeFdzXgE,https://www.youtube.com/watch?v=MuqJeFdzXgE&t=...,7
4,ne var ne yok,6826,6828,94KUKlC1rlc,https://www.youtube.com/watch?v=94KUKlC1rlc&t=...,7
5,ne var ne yok,97,99,S6wCHPZLjsw,https://www.youtube.com/watch?v=S6wCHPZLjsw&t=97s,7
6,ne var ne yok,5868,5870,cslXqrcH5qg,https://www.youtube.com/watch?v=cslXqrcH5qg&t=...,7
7,bir fikrin var mı,2480,2482,lNwXQrpMfhM,https://www.youtube.com/watch?v=lNwXQrpMfhM&t=...,7
8,bir fikrin var mı,5895,5897,sMEBFgBFmy8,https://www.youtube.com/watch?v=sMEBFgBFmy8&t=...,7
9,bir fikrin var mı,1318,1320,cXAaYu0JM_s,https://www.youtube.com/watch?v=cXAaYu0JM_s&t=...,7


In [73]:
search_list = df_word_group_time_loc.iloc[:,0].unique()
len(search_list)

11

In [74]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["search_string","start_time","end_time","sentence","video_id","video_url"])
df_link_default

,search_string,start_time,end_time,sentence,video_id,video_url
0,repeat,5,7,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s


In [75]:
#(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [76]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
seq_num = 0  # option
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
        df_result_repeat.reset_index(drop=True, inplace=True)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"]) 
    df_result_repeat_var.loc[0,["search_string"]] = f"{word_group}"  # option for search string
    df_result_repeat_var.loc[0,["count"]] = df_result_repeat.loc[0,"count"] # option for count num     
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=0)

df_all_join_result.reset_index(drop=True, inplace=True)
df_all_join_result["web_format"] = f"[play_lms_videos video_id="+df_all_join_result["video_id"].map(str)+" start_time="+df_all_join_result["start_time"].map(str)+" end_time="+df_all_join_result["end_time"].map(str)+"]"
df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx", index=False)


In [77]:
df_all_join_result

,video_id,start_time,end_time,search_string,count,web_format
0,"KjOlhIa9X2Y,XWd602XfKh0,OckqrD0x_r8,MuqJeFdzXg...","208,2182,3000,2763,6826,97,5868,3000,5,3000,5,...","210,2184,3002,2765,6828,99,5870,3002,8,3002,8,...",ne var ne yok,7.0,"[play_lms_videos video_id=KjOlhIa9X2Y,XWd602Xf..."
1,"lNwXQrpMfhM,sMEBFgBFmy8,cXAaYu0JM_s,7XVwNBlTIW...","2480,5895,1318,6755,5491,2239,4020,2480,5,2480...","2482,5897,1320,6758,5494,2241,4023,2482,8,2482...",bir fikrin var mı,7.0,"[play_lms_videos video_id=lNwXQrpMfhM,sMEBFgBF..."
2,"g81mNCrBnO4,eGGS2bYtnLE,UJBXM9wxjlI,8_VkxmHeMA...","775,145,1706,2650,1009,5576,2346,775,5,775,5,7...","777,147,1708,2652,1011,5578,2349,777,8,777,8,7...",başka bir şey yok,7.0,"[play_lms_videos video_id=g81mNCrBnO4,eGGS2bYt..."
3,"XcJfbADgIhs,DmHNlqpCvvE,kKmkQ42kXXI,xIjA_1JQF8...","4549,5695,531,4673,4868,4868,5,4868,5,4868,5","4551,5697,533,4676,4871,4871,8,4871,8,4871,8",bir şey sorabilir miyim,5.0,"[play_lms_videos video_id=XcJfbADgIhs,DmHNlqpC..."
4,"r4o0Vg6NQNc,H2h6mFizN7w,NoeaLFiCFkc,MhHCQ_cb3T...","2268,2969,1217,7957,1980,2268,5,2268,5,2268,5","2271,2971,1220,7959,1982,2271,8,2271,8,2271,8",ben de seni seviyorum,5.0,"[play_lms_videos video_id=r4o0Vg6NQNc,H2h6mFiz..."
5,"g81mNCrBnO4,uLIQAbifOME,lYSwby37FwQ,b282Syiyye...","4856,577,878,2324,577,5,577,5,577,5","4858,579,880,2326,579,8,579,8,579,8",önemli bir şey değil,4.0,"[play_lms_videos video_id=g81mNCrBnO4,uLIQAbif..."
6,"259ZLluIkyk,8WHuCHRHWqM,Ca9zvaqTlXk,jGMsIgTA_w...","656,560,4264,1602,4264,5,4264,5,4264,5","658,566,4266,1604,4266,8,4266,8,4266,8",ne olduğunu biliyor musun,4.0,"[play_lms_videos video_id=259ZLluIkyk,8WHuCHRH..."
7,"i_fb_jHCUE8,sG77A55HmHs,LC2QScFT5QI,TPShfpgrg6...","3396,1955,329,5699,5699,5,5699,5,5699,5","3399,1957,331,5701,5701,8,5701,8,5701,8",burada ne işin var,4.0,"[play_lms_videos video_id=i_fb_jHCUE8,sG77A55H..."
8,"Odbd1stkcRk,NV_G6wX2xtQ,NzHqMh9dfh0,evVtrLqloL...","7746,1085,534,1002,1002,5,1002,5,1002,5","7748,1087,536,1005,1005,8,1005,8,1005,8",bir şey mi var,4.0,"[play_lms_videos video_id=Odbd1stkcRk,NV_G6wX2..."
9,"KwysdCxTSVw,JxcQrfHYKqI,J3ZzEixVXKs,J3ZzEixVXK...","2025,2861,4835,4835,5,4835,5,4835,5","2028,2864,4837,4837,8,4837,8,4837,8",sizin için ne yapabilirim,3.0,"[play_lms_videos video_id=KwysdCxTSVw,JxcQrfHY..."


#### Copy Move And Delete

In [78]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx")
output_file

['Turkish_5000_Word_Youtube_Talk_Time_Join_Fourgram.xlsx']

In [79]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [80]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

In [ ]:
#df_test["web_format"] = f"[play_lms_videos video_id="+df_test["video_id"].map(str)+" start_time="+df_test["start_time"].map(str)+" end_time="+df_test["end_time"].map(str)+"]"
#df_test